In [7]:
from imutils import paths
import face_recognition
import pickle
import cv2
import os

imagePaths = list(paths.list_images('D:\photos')) 
knownEncodings = []
knownNames = []

for (i, imagePath) in enumerate(imagePaths):
    
    name = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb,model='hog')
    encodings = face_recognition.face_encodings(rgb,boxes)
    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(name)
data = {"encodings": knownEncodings, "names": knownNames}

f = open("face_enc", "wb")
f.write(pickle.dumps(data))
f.close()

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
!pip install imutils

  Using cached imutils-0.5.4-py3-none-any.whl


In [6]:
!pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Running setup.py clean for dlib
Failed to build dlib
    Running setup.py install for dlib: started
    Running setup.py install for dlib: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\SHWETHA S P\anaconda3\envs\sdp\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\SHWETHA S P\\AppData\\Local\\Temp\\pip-install-ywu3pmhg\\dlib_5443ce0fc16c416db4d0cdb89c70c2f7\\setup.py'"'"'; __file__='"'"'C:\\Users\\SHWETHA S P\\AppData\\Local\\Temp\\pip-install-ywu3pmhg\\dlib_5443ce0fc16c416db4d0cdb89c70c2f7\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\SHWETHA S P\AppData\Local\Temp\pip-wheel-254rfn0j'
       cwd: C:\Users\SHWETHA S P\AppData\Local\Temp\pip-install-ywu3pmhg\dlib_5443ce0fc16c416db4d0cdb89c70c2f7\
  Complete output (8 lines):
  running bdist_wheel
  running build
  running build_py
  package init file 'tools\p

In [4]:
!pip install opencv-python

In [ ]:
import face_recognition
import imutils
import pickle
import time
import cv2
import os

import firebase_admin
from firebase_admin import credentials, db

cred = credentials.Certificate(r"C:\Users\SHWETHA S P\Downloads\face.json")
firebase_admin.initialize_app(cred,{"databaseURL":"https://face-recognition-a7783-default-rtdb.firebaseio.com/"})
store=db.reference("/")

cascPathface = os.path.dirname(cv2.__file__)+"/data/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPathface)
data = pickle.loads(open('face_enc',"rb").read())
print("Streaming started")
video_capture = cv2.VideoCapture(0)
while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(60, 60),flags=cv2.CASCADE_SCALE_IMAGE)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb)
    names = []
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"],encoding)
        name = "Unknown"
        if True in matches:
            matchedIdxs = [i for (i,b) in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get)
        names.append(name)
        for((x,y,w,h),name)in zip(faces, names):
            cv2.rectangle(frame,(x,y),(x + w,y + h),(0, 255, 0),2)
            cv2.putText(frame, name , (x,y), cv2.FONT_HERSHEY_SIMPLEX,
                       0.75, (0, 255, 0),2)
    cv2.imshow("Frame", frame)
    if(name=="Unknown"):
        store.update({"person":"Unknown"})
    else:
        store.update({"person":name})
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

Streaming started
